<a href="https://colab.research.google.com/github/Tejaswini170104/DA6401-A2/blob/main/PartA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 01

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import torchvision.transforms as transforms

class FlexibleCNN(pl.LightningModule):
    def __init__(self, input_channels=3, num_classes=10, conv_filters=[32, 64, 128, 256, 512],
                 kernel_size=3, activation=F.relu, dense_neurons=256, input_size=224):
        super(FlexibleCNN, self).__init__()

        self.activation = activation
        layers = []
        in_channels = input_channels

        # Adding 5 convolutional blocks
        for out_channels in conv_filters:
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, padding=1))
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(2, 2))
            in_channels = out_channels

        self.conv_layers = nn.Sequential(*layers)

        # Compute final feature map size after 5 max-pool layers
        final_size = input_size // (2 ** len(conv_filters))

        # Fully connected layers
        self.fc1 = nn.Linear(conv_filters[-1] * final_size * final_size, dense_neurons)
        self.fc2 = nn.Linear(dense_neurons, num_classes)

        # Compute parameters and computations
        self.total_parameters = self.compute_parameters(conv_filters, kernel_size, dense_neurons, num_classes)
        self.total_computations = self.compute_computations(conv_filters, kernel_size, input_size)

        # Define preprocessing transform for iNaturalist dataset
        self.transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def forward(self, x):
        x = self.conv_layers(x)
        x = torch.flatten(x, start_dim=1)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x

    def compute_parameters(self, conv_filters, kernel_size, dense_neurons, num_classes):
        total_params = 0
        in_channels = 3  # Input image channels

        # Compute parameters for convolution layers
        for out_channels in conv_filters:
            total_params += (in_channels * kernel_size * kernel_size + 1) * out_channels
            in_channels = out_channels

        # Compute parameters for fully connected layers
        total_params += (conv_filters[-1] * (224 // (2**5)) * (224 // (2**5))) * dense_neurons + dense_neurons
        total_params += dense_neurons * num_classes + num_classes

        return total_params

    def compute_computations(self, conv_filters, kernel_size, input_size):
        total_computations = 0
        in_channels = 3  # Input image channels
        feature_map_size = input_size

        # Compute computations for convolution layers
        for out_channels in conv_filters:
            feature_map_size //= 2  # Max-pooling reduces size by half
            total_computations += out_channels * feature_map_size * feature_map_size * (in_channels * kernel_size * kernel_size)
            in_channels = out_channels

        return total_computations

# Example usage:
model = FlexibleCNN(input_channels=3, num_classes=10, conv_filters=[32, 64, 128, 256, 512],
                    kernel_size=3, activation=F.relu, dense_neurons=256, input_size=224)
print(model)
print(f"Total Parameters: {model.total_parameters}")
print(f"Total Computations: {model.total_computations}")

FlexibleCNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

Question 02